In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Send the data to the sheet

In [5]:
sql = 'select * from crowdsdb.dbo.vw_consolidated_socialdistancing'

In [6]:
consolidated_sql = (pd.read_sql(con = engine, sql = sql)
                    .fillna(value = np.nan, axis = 1))

In [7]:
consolidated_sql.tail()

,source_survey,encounter_datetime,gispropnum,reported_as,site_id,site_desc,park_borough,location_adddesc,city_agency,encounter_type,amenity,patroncount,police_precinct,police_boro_com,communityboard
31581,DPR Crowds,2020-06-02 10:58:56,Q131,Q131,Q131-01,Frank Principe Park-Virginia Principe Playground,Queens,NaN,DPR,NaN,Soccer field,80.0,104,Queens North,405
31582,DPR Crowds,2020-06-02 18:46:01,Q004,Q004,Q004,Astoria Park,Queens,NaN,DPR,NaN,Soccer field,100.0,114,Queens North,401
31583,DPR Crowds,2020-06-02 17:47:38,Q392,Q392,Q392-ZN01,Idlewild Park Zone 1,Queens,NaN,DPR,NaN,Open field/multi-purpose play area,20.0,105,Queens South,413
31584,DPR Crowds,2020-06-02 17:32:52,Q207,Q207,Q207,MacDonald Park,Queens,NaN,DPR,NaN,Chess area,20.0,112,Queens North,406
31585,DPR Crowds,2020-06-02 17:10:48,X147A,X147A,X147A-ZN01,Starlight Park Zone 1,Bronx,NaN,DPR,NaN,Soccer field,30.0,43,Bronx,209


In [8]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [9]:
sheet = client.open('consolidated_social_distancing_data')

In [10]:
ws = sheet.worksheet('Sheet1')

In [11]:
set_with_dataframe(ws, consolidated_sql, include_index = False, 
                   include_column_header = True, resize = True, allow_formulas = False)